In [1]:
# C:\Users\wwsch\anaconda3\python.exe
# Name: William Schlough (wschloug)
# Group Members: Junmo, Mikey, Wenyu

########################################################################
# FastAreader
# provided by Dr. B
########################################################################
import sys


class FastAreader:

    def __init__(self, fname=''):
        '''contructor: saves attribute fname '''

        self.fname = fname

    def doOpen(self):
        if self.fname is '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta(self):

        header = ''
        sequence = ''

        with self.doOpen() as fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>'):
                line = fileH.readline()
            header = line[1:].rstrip()

            for line in fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence


In [2]:
########################################################################
# CommandLine
# provided by Dr. B
########################################################################

class CommandLine():
    '''
    Handle the command line, usage and help requests.

    CommandLine uses argparse,
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination exception Usage.

    attributes:
    all arguments received from the commandline using .add_argument will be
    available within the .args attribute of an object instantiated from CommandLine.
    For example, if thisCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then thisCommandLine.args.requiredbool will
    name that option.

    '''

    def __init__(self, inOpts=None):
        '''
        CommandLine constructor.

        Implement a parser to interpret the command line argv string using argparse.
        '''

        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='python missingMotif.py -minMotif int -maxMotif int -cutoff int < input.fa > output.out'
        )

        # options as specified by assignment
        self.parser.add_argument('-minMotif', type=int, choices=range(3, 9), action='store', default=3,
                                 help='minimum motif size in integer format')
        self.parser.add_argument('-maxMotif', type=int, choices=range(3, 9), action='store', default=8,
                                 help='maximum motif size in integer format')
        self.parser.add_argument('-cutoff', type=int, action='store', default=0,
                                 help='z score cutoff in integer format')

        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)


class Usage(Exception):
    '''
    Signal a Usage error, evoking a usage statement and eventual exit when raised.
    '''

    def __init__(self, msg):
        self.msg = msg


In [3]:
import math # needed for sqrt -- realized i could use **
########################################################################
# Genome
#
########################################################################
class Genome:
    '''

    class genome contains methods that are used to find the z-score, expected value, and count of all motifs

    setup --------------
    initialization
    z-score
    expected value
    motifcount / kmer list -- dict?

    way to get reverse compliment easily -- bme 160 method

    '''



    def __init__(self, seq, N, min, max, cutoff):
        '''


        seq -- input sequences taken from fastaReader
        N -- genome size (length of all sequences)
        min -- minimum motif size to look at (4?)
        max -- maximum motif size to look at (8)
        cutoff -- cutoff of z value (should be negative values -- more negative more deviation)

        '''

        self.seq = seq
        self.min = min
        self.max = max
        self.cutoff = cutoff

        self.kmerCounts = dict() # keep track of kmer + times found in seq (along with counting for reverse compliment too)

        self.N = N  # genome size

        for sequence in self.seq: # for each sequence
            for k in range(1, self.max+1): # for each motif
                for i in range(0, len(sequence) - k + 1): # find all motifs of that size
                    kmer = ''.join(sequence[i:i + k]) # kmer cut from i to k

                    # only look at top strand
                    if kmer in self.kmerCounts: # if kmer already exists
                        self.kmerCounts[kmer][0] += 1 # add one to the count
                    else:
                        self.kmerCounts[kmer] = [1]
                        self.kmerCounts[self.reverseSeq(kmer)] = self.kmerCounts[kmer] # set new key pointing to same

                    #if self.reverseSeq(kmer) in self.kmerCounts: # check if the reverse compliment is in dictionary
                        #self.kmerCounts[kmer] = self.kmerCounts[self.reverseSeq(kmer)]
                        #self.kmerCounts[kmer][0] += 1 # add one to the count of specified kmer

                    #elif kmer in self.kmerCounts: # else if its already in dictionary
                        #self.kmerCounts[kmer][0] += 1 # add 1 to the count of specified kmer

                    #else: # otherwise
                        #self.kmerCounts[kmer] = [1] # add it as a key with starting value 1 to dictionary

    def reverseSeq(self, sequence):
        '''

        used to find reverse compliment -- method taken from bme 160, very useful
        returns the reverse compliment

        https://www.w3schools.com/python/ref_string_translate.asp
        https://www.programiz.com/python-programming/methods/string/maketrans

        '''

        #(sequence.translate(str.maketrans("ATCG", "TAGC"))[::-1])
        return sequence.translate(str.maketrans("ATCG", "TAGC"))[::-1]

    def eVal(self, kmer):
        '''

        using markovian model -- left + right size + middle kmer are used to calculate escore
        calculates and returns the expected value of a specific motif appearing

        '''
        tail = len(kmer) - 1 # like middle
        left = kmer[:-1] # left kmer
        right = kmer[1:] # right kmer

        # consider counts of left side + right side of motif
        numerator = self.kmerCounts[left][0] * self.kmerCounts[right][0]

        # consider counts of middle part of motif
        denominator = self.kmerCounts[kmer[1:tail]][0]

        # divide the counts of left + right side of motif by the counts of middle part of motif
        return numerator / denominator

    def zScore(self, kmer):
        '''

        takes in -- a kmer that has eVal called on it to calculate the mean, calculates Pr(K),
                    then calculates the stand deviation, and lastly calculates a z-score
        returns a z-score

        For this assignment, we will use Z-scores.
        These will be negative since we are looking for underrepresented sequences (this cutoff needs to be a program option too)

        "A score that indicates how many standard deviations a value is above or below the mean."

        '''
        mean = self.eVal(kmer)
        N = self.N # "n = genome size (approximately)"
        p = mean / N # Pr(K) = E(K)/N -- N being length of genome, and we use eval() method for E(K)

        standardDev = math.sqrt(N * p * (1 - p))  # sqrt(np(1-p)


        zscore = (self.kmerCounts[kmer][0] - mean) / standardDev # calculate z score
        return zscore




    def kmerList(self):
        '''

        returns a sorted list containing the (kmer, reverse compliment, counts of both, e value, z score, motif size)
        the list is sorted first by motif size, and then by z score

        '''
        kmax = 8  # due to program requirements -- "We need only consider sequence motifs that are up to about 8 in length"
        kmerList = list()
        check = list()

        # check all keys
        for kmer in self.kmerCounts.keys():
            # kmer between min and max
            # reverse not yet added (using listcheck to see if kmer has been added yet or not)
            # zscore less than (but not including) cutoff
            if self.min <= len(kmer) <= self.max and \
                    self.reverseSeq(kmer) not in check and \
                    self.zScore(kmer) < self.cutoff:

                k = kmax - len(kmer) # max motif size
                # sequence: reverse     count      Expect Zscore
                kmerList.append([kmer,
                                 self.reverseSeq(kmer),
                                 self.kmerCounts[kmer][0],
                                 self.eVal(kmer),
                                 self.zScore(kmer),
                                 k]) # motif size
                check.append(kmer)

        # per program requirements -- "sorted by Zscore within motif-Size"
        # such that the longest motifs are at the beginning of the report and,
        # within motifs of a given size, print the most significant motifs first.
        # Print the k-mer pairs in alpha order.
        # ( AAA:TTT and not TTT:AAA)
        # kmerList sorted by (motif-size, then by z-score)
        return sorted(kmerList, key=lambda kmer:(kmer[5], kmer[4]))


def main(inFile, myCommandLine=None):
    '''
    Implement the Usage exception handler that can be raised from anywhere in process.

    used group member mikey's way of coding inFile argument into main
    '''
    if myCommandLine is None:
        myCommandLine = CommandLine()  # read options from the command line
    else:
        myCommandLine = CommandLine(myCommandLine)  # interpret the list passed from the caller of main as the commandline.

    seqList = []
    N = 0 # will keep track of genome size by counting length of each sequence passed by FastAreader,

    myReader = FastAreader(inFile) # <------  input .fa file here
    for head, seq in myReader.readFasta():
        seqList.append(seq)
        N += len(seq)

    myMotifs = Genome(seqList,
                      N, # then passed as a var in myMotifs = Genome() initialization
                      myCommandLine.args.minMotif,
                      myCommandLine.args.maxMotif,
                      myCommandLine.args.cutoff)

    kmerList = myMotifs.kmerList()
    print("N: {}".format(N))
    print("sequence:reverse \t\t count \t\t Expect \t\t Zscore")
    for kmer in kmerList:  # prints each kmer with respective data (in formatted order -- done through kmerList())
        print('{0:8}:{1:8} \t\t {2:0d} \t\t {3:0.2f} \t\t {4:0.2f}'.format(kmer[0], # kmer
                                                                           kmer[1][::-1], # reverse compliment kmer [::-1] to reverse the sequence to fit the output parameters
                                                                           kmer[2], # count
                                                                           kmer[3], # eValue
                                                                           kmer[4])) # z-score


if __name__ == "__main__":
    main(inFile='Arthrospira-platensis-NIES-39.fna',                     # <--------- put input filename here
         myCommandLine=['-minMotif= 3',
                        '-maxMotif= 8',
                        '-cutoff= -4']) # <------- put your min, max, and cutoff here

N: 6788435
sequence:reverse 		 count 		 Expect 		 Zscore
GCGATCGC:CGCTAGCG 		 7683 		 31752.94 		 -135.39
AATTAATT:TTAATTAA 		 558 		 2580.47 		 -39.82
AAAATTTT:TTTTAAAA 		 501 		 2185.49 		 -36.04
TTAATTAA:AATTAATT 		 537 		 2205.76 		 -35.54
AAATATTT:TTTATAAA 		 365 		 1705.30 		 -32.46
ATAATTAT:TATTAATA 		 364 		 1681.01 		 -32.13
TTTTAAAA:AAAATTTT 		 298 		 1251.69 		 -26.96
CAAATTTG:GTTTAAAC 		 245 		 1158.42 		 -26.84
TAATATTA:ATTATAAT 		 296 		 1222.71 		 -26.50
TAAATTTA:ATTTAAAT 		 413 		 1380.29 		 -26.04
GATTAATC:CTAATTAG 		 390 		 1342.07 		 -25.99
TGATATCA:ACTATAGT 		 300 		 1184.38 		 -25.70
TATTAATA:ATAATTAT 		 385 		 1310.43 		 -25.57
ATCGCGAT:TAGCGCTA 		 294 		 1155.39 		 -25.34
AATATATT:TTATATAA 		 255 		 1051.57 		 -24.57
ATTTAAAT:TAAATTTA 		 195 		 945.67 		 -24.41
GTTTAAAC:CAAATTTG 		 269 		 1057.29 		 -24.25
CCCTAGGG:GGGATCCC 		 232 		 997.07 		 -24.23
CAATATTG:GTTATAAC 		 230 		 971.23 		 -23.79
ACAATTGT:TGTTAACA 		 115 		 750.77 		 -23.20
CATTAATG:GTAATTAC 		 165

__CODE MARKDOWN CELL__

__Schlough_William_missingMotif.ipynb__ contains ----------------- 3 classes - FastAreader, CommandLine, and Genome -- along with a main

__FastAreader__ and __CommandLine__ were provided by Dr. B, and only CommandLine was slightly changed to fit the program requirements

__Genome__ contains class genome contains methods that are used to find the z-score, expected value, and count of all motifs, along with a sorted list that can be printed to an output file

def __ __init__ __(self, seq, N, min, max, cutoff) -- initialization of user inputs occurs here

def __reverseSeq__(self) -- quick way to get reverse compliment - taken from BME 160

def __eVal__(self, kmer) -- using markovian model -- left + right size + middle kmer are used to calculate eScore, calculates and returns the expected value of a specific motif appearing

def __zScore__(self, kmer) -- takes in -- a kmer that has eVal called on it to calculate the mean, calculates Pr(K), then calculates the stand deviation, and lastly calculates a z-score and returns that z-score

def __kmerList__(self) -- returns a sorted list containing the (kmer, reverse compliment, counts of both, e value, z score, motif size), the list is sorted first by motif size, and then by z score

__main__ -- for this assignment, we print the sorted list in order of motif size, then by zScore. (variable N is used to keep track of genome length, and is then passed onto Genome() instantiation, becoming self.N)

__INSPECTION MARKDOWN CELL__

DoneBy: Junmo Sung

It was well explained with the markdown cells and docstrings.
But it seems there isn't a code for the case of denominator of 0 (if there is, I cannot find the explanation about it).
Other than that it is well written and the outcomes look fine.



Inpsector: Michael
Inspectee: Will

Change counting method to not have to count the reverse strand by using the ditionary structre where the reverse complement motifs point to the same value.

Don't have to check if std != 0 in your zscore function since it can't be with the way you've counted your motifs

Rest looks good, definitely stealing your reverse complement method

